<a href="https://colab.research.google.com/github/kvanarsd/ASTR-19-Final/blob/main/UDF_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install sep

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
# read image into standard 2-d numpy array
path = "/content/drive/MyDrive/ASTR19_Final/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
# open file
image = fits.open(path)
# turn image into data array
data = image[0].data
data = data.byteswap().newbyteorder()

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig("Img.png")

In [ ]:
# measure a spatially varying background on the image
# background subtraction
bkg = sep.Background(data)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("Background.png")

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("BackgroundNoise.png")

In [ ]:
# subtract the background
data_sub = data - bkg

In [ ]:
# detect objects
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
len(objects)

Found 8643 sources

In [ ]:
# plotting all the objects found
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig("CircledData.png")

In [ ]:
# available fields
objects.dtype.names

In [ ]:
# perform circular aperture photometry with 3 px radius
# how bright a source is
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
def histogram_plot(
    flag_save=True, #save the figure?
    xlabel='Bins', #x-axis label
    ylabel='Frequency', #y-axis label
    lcolor='red', #line color
    fs=14, #font size
    fname='FluxHistogram.png'):
    hist, bins = plt.subplots(1,1,figsize=(4,9))

    plt.hist(flux,facecolor=lcolor,bins=10000,edgecolor='black',alpha=.5)
    plt.xscale('symlog')
    #plt.xscale('log')

    bins.set_xlabel(xlabel,fontsize=fs)
    bins.set_ylabel(ylabel,fontsize=fs)

    if(flag_save):
        plt.savefig(fname,bbox_inches='tight',dpi=400)

    plt.show()

histogram_plot()

In [ ]:
# median
np.median(flux)

In [ ]:
# mean
mean = np.mean(flux)

# standard deviation
std = np.std(flux)

In [ ]:
# outier
outlier = np.max(flux)

# the largest outlier is:
(outlier - mean) / std
# standard deviations away from the mean

The largest outlier is 87.31249046216263 standard deviations away from the mean

In [ ]:
# read image into standard 2-d numpy array
path125 = "/content/drive/MyDrive/ASTR19_Final/hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
path160 = "/content/drive/MyDrive/ASTR19_Final/hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"
# open file
image125 = fits.open(path125)
image160 = fits.open(path160)

# turn image into data array
data = image[0].data
data125 = image125[0].data
data160 = image160[0].data

In [ ]:
def rescale_image(data):
  pdata_tmp = data.copy()
  m = np.nanmean(pdata_tmp)
  vplmin = m/2.
  vpmin = np.log10(vplmin)
  vpmax = np.log10(m * 100.)
  pdata_tmp[pdata_tmp<vplmin] = vplmin
  pdata_tmp = np.log10(pdata_tmp)
  return pdata_tmp,vpmin, vpmax

In [ ]:
def fits_quicklook(data, fnx=10, fny=10):
  f = plt.figure(figsize=(fnx,fny))
  pdata_tmp, vpmin, vpmax = rescale_image(data)
  plt.imshow(pdata_tmp, vmin = vpmin, vmax = vpmax)

In [ ]:
data_105_res, d105min, d105max = rescale_image(data)
data_125_res, d125min, d125max = rescale_image(data)
data_160_res, d160min, d160max = rescale_image(data)

In [ ]:
data_105_res[data_105_res<d105min] = d105min
data_105_res[data_105_res>d105max] = d105max
data_125_res[data_125_res<d125min] = d125min
data_125_res[data_125_res>d125max] = d125max
data_160_res[data_160_res<d160min] = d160min
data_160_res[data_160_res>d160max] = d160max

In [ ]:
rgb = np.zeros((data_105_res.shape[0], data_105_res.shape[1], 3))
rgb[:,:,0] = (data_105_res-d105min) / (d105max-d105min)
rgb[:,:,1] = (data_125_res-d125min) / (d125max-d125min)
rgb[:,:,2] = (data_160_res-d160min) / (d160max-d160min)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(20,20))
ax.axis('off')
ax.imshow(rgb)
plt.savefig("3-color-false.png", bbox_inches='tight',pad_inches=0, dpi=600)